<a href="https://colab.research.google.com/github/pyoph/ProjetHDP/blob/main/Copie_de_projetHDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import cvxpy as cp
import numpy as np

In [157]:
#Premiers essais relaxation convexe (p,q) SBM : X_i,i = 1


np.random.seed(1)

#Paramètres
n = 10

#Définitions des seuils
p = 0.8
q = 0.08
l = 0


J3 = np.array([[1,1,1],[1,1,1],[1,1,1]])

#Définition de la matrice Z à estimer

Z = np.random.choice([0, 1], size=(n, n))

for i in range(n) :
  Z[i][i] = 1




#Tuning parameter

for i in range(n) :
  for j in range(n) :
    l += Z[i][j]



#Calcul de la matrice d'adjacence à partir des tirages de p et de q
A = np.zeros((n,n))

#Définition de la matrice A observée

for i in range(n):
  for j in range(n) :
    if (Z[i][j] == 0) : A[i][j] = np.random.choice([0,1],p = [1 -p,p])
    else : A[i][j] = np.random.choice([0,1],p = [1 -q,q])



print(A)

C = []
b = []
for i in range(n):
    b.append(1)

M = np.zeros((n,n))


#Concaténation des matrices élémentaires pour faire le produit scalaire
for  i in range(n) :
  M[i][i] = 1
  C.append(M)
  M = np.zeros((n,n))

print(C)


# Define and solve the CVXPY problem.
# Create a symmetric matrix variable.
X = cp.Variable((n,n), symmetric=True)
# The operator >> denotes matrix inequality.
constraints = [X >> 0]
#constraints += [
#    cp.trace(C[i] @ X) <= b[i] for i in range(n)
#]
constraints += [
    cp.diag(X) <= np.ones(n)
]


constraints += [
    cp.sum(X) == l
]


prob = cp.Problem(cp.Minimize(cp.trace(A @ X)),
                  constraints)
prob.solve()

# Print result.
print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)

[[0. 0. 1. 1. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 1. 1. 0. 1. 1.]
 [1. 0. 0. 1. 0. 0. 1. 1. 0. 1.]
 [1. 1. 0. 0. 0. 0. 0. 1. 1. 1.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 1.]
 [1. 0. 1. 1. 0. 0. 0. 1. 0. 1.]
 [0. 0. 1. 1. 0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0. 1.]
 [0. 1. 1. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
[array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0

ValueError: Incompatible dimensions (1, 10) (5, 1)

In [224]:
import cvxpy as cp
import numpy as np

# Define dimensions and data
n = 10
rho = 1.0
MAX_ITERS = 100

# Initialize problem.
x = cp.Variable(shape=n)
f = cp.matmul(cp.transpose(A)[0,:], x)

# Constraints
constraints2 = [
    x >= 0 ,
    cp.sum(x) <= l/n,
    x[0] <= 1
]

# Solve with CVXPY.
problem_cvxpy = cp.Problem(cp.Minimize(f), constraints2)
problem_cvxpy.solve()
print("Optimal value from CVXPY: {}".format(problem_cvxpy.value))
optimal_x = x.value

print("Optimal x from method of multipliers: {}".format(optimal_x))


# Solve with method of multipliers.
resid = cp.matmul(M, x) - b
y = cp.Parameter(shape=(n))
y.value = np.zeros(n)

# Augmented Lagrangian
aug_lagr = f + cp.matmul(cp.transpose(y), resid) + (rho/2) * cp.sum_squares(resid)

for t in range(MAX_ITERS):
    problem_multipliers = cp.Problem(cp.Minimize(aug_lagr), constraints2)
    problem_multipliers.solve()

    # Check if the problem is feasible
    if problem_multipliers.status not in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
        print("The problem is not feasible.")
        break

    # Update Lagrange multiplier
    y.value += rho * (resid.value if resid.value is not None else np.zeros(n))

# Get the optimal solution
optimal_x = x.value

print("Optimal x from method of multipliers: {}".format(optimal_x))


Optimal value from CVXPY: 1.2064913343991727e-10
Optimal x from method of multipliers: [7.82354935e-01 8.19605866e-01 3.01622834e-11 3.01622834e-11
 8.19605866e-01 3.01622834e-11 8.19605866e-01 8.19605866e-01
 8.19605866e-01 3.01622834e-11]
Optimal x from method of multipliers: [ 1.00000000e+00 -0.00000000e+00  1.00013542e-30  1.00013542e-30
 -0.00000000e+00  1.00013542e-30 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  1.00013542e-30]


In [227]:
#Calculs précédents insérés dans une fonction
def caculeVecteurXioptimal(i) :
  x = cp.Variable(shape=n)
  f = cp.matmul(cp.transpose(A)[i,:], x)
  # Constraints
  constraints2 = [
    x >= 0 ,
    cp.sum(x) <= l/n,
    x[i] <= 1
   ]

# Solve with CVXPY.
  problem_cvxpy = cp.Problem(cp.Minimize(f), constraints2)
  problem_cvxpy.solve()
  optimal_x = x.value



  # Solve with method of multipliers.
  resid = cp.matmul(M, x) - b
  y = cp.Parameter(shape=(n))
  y.value = np.zeros(n)

  # Augmented Lagrangian
  aug_lagr = f + cp.matmul(cp.transpose(y), resid) + (rho/2) * cp.sum_squares(resid)

  for t in range(MAX_ITERS):
    problem_multipliers = cp.Problem(cp.Minimize(aug_lagr), constraints2)
    problem_multipliers.solve()

    # Check if the problem is feasible
    if problem_multipliers.status not in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
        print("The problem is not feasible.")
        break

    # Update Lagrange multiplier
    y.value += rho * (resid.value if resid.value is not None else np.zeros(n))

# Get the optimal solution
  optimal_x = x.value
  return optimal_x

optimal_x = caculeVe!cteurXioptimal(1)
print("Optimal x from method of multipliers: {}".format(optimal_x))


Optimal x from method of multipliers: [ 1.00000000e+00 -0.00000000e+00 -0.00000000e+00  1.00013542e-30
  1.00013542e-30 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  1.00013542e-30 -0.00000000e+00]


In [230]:
#Création de la matrice X à partir du calcul de chacun des vecteurs X

Xopt = np.zeros(n)

for i in range(n):
    vecteur_i_optimal = caculeVecteurXioptimal(i)  # Assurez-vous que la fonction renvoie un vecteur de dimension n
    Xopt = np.vstack((Xopt, vecteur_i_optimal))

# Supprimer la première ligne de zéros ajoutée lors de l'initialisation
Xopt = Xopt[1:]

# Xopt est maintenant une matrice où chaque ligne est un vecteur optimal
print("Matrice Xopt :\n", Xopt)



Matrice Xopt :
 [[ 1.00000000e+00 -0.00000000e+00  1.00013542e-30  1.00013542e-30
  -0.00000000e+00  1.00013542e-30 -0.00000000e+00 -0.00000000e+00
  -0.00000000e+00  1.00013542e-30]
 [ 1.00000000e+00 -0.00000000e+00 -0.00000000e+00  1.00013542e-30
   1.00013542e-30 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
   1.00013542e-30 -0.00000000e+00]
 [ 1.00000000e+00  1.00013542e-30 -0.00000000e+00 -0.00000000e+00
  -0.00000000e+00  1.00013542e-30  1.00013542e-30 -0.00000000e+00
   1.00013542e-30 -0.00000000e+00]
 [ 1.00000000e+00 -0.00000000e+00  1.00013542e-30 -0.00000000e+00
  -0.00000000e+00  1.00013542e-30  1.00013542e-30  1.00013542e-30
  -0.00000000e+00 -0.00000000e+00]
 [ 1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  -0.00000000e+00 -0.00000000e+00 -0.00000000e+00  1.00013542e-30
  -0.00000000e+00 -0.00000000e+00]
 [ 1.00000000e+00  1.00013542e-30 -0.00000000e+00 -0.00000000e+00
  -0.00000000e+00 -0.00000000e+00 -0.00000000e+00  1.00013542e-30
   1.00013542e-30